In [1]:
import pandas as pd
# Project level modules
from modules import database_connection, sql_statements
from modules.database_connection import (
    postgresql_connection, get_table_data_types, dataframe_to_csv,
    execute_sql_statement, get_descriptive_statistics
)

# Make PostgreSQL database connection

# Table Names

In [2]:
table_names = ['flights', 'flights_test', 'fuel_comsumption', 'passengers']

In [3]:
cat_features_flights = ['fl_date', 'mkt_unique_carrier']

In [4]:
con = postgresql_connection()

Connected


In [5]:
frames = []
for name in table_names:
    # Generate list of dataframes of table datatypes
    df = get_table_data_types(connection=con,
                                table_name=name)
    frames.append()

In [15]:
for frame in frames:
    frame.insert(loc=0, column='table_name', value=frame.columns.name)

In [16]:
frames[0]

flights,table_name,column_name,data_type
0,flights,fl_date,text
1,flights,mkt_unique_carrier,text
2,flights,branded_code_share,text
3,flights,mkt_carrier,text
4,flights,mkt_carrier_fl_num,integer
5,flights,op_unique_carrier,text
6,flights,tail_num,text
7,flights,op_carrier_fl_num,integer
8,flights,origin_airport_id,integer
9,flights,origin,text


In [17]:
# Concatenate frames on the columns
data_types_df = pd.concat(frames, axis=1)

In [19]:
dataframe_to_csv(data_types_df, '../data/table_data_types.csv')